In [4]:
import pandapower as pp
import numpy as np
from pandapower import plotting as plot
import matplotlib.pyplot as plt
import seaborn
import pandas as pd


#First Try on Balearic Islands
hv=230 #in kV
mv=132 #in kV
lv=66  #in kV


#XLPE 230kV 630mm2 Cables
R_230=0.0393
X_230=0.086
Max_230=0.95
C_230=0.13e-3

#XLPE 132kV 240mm2 Cables

R_132=0.0973
X_132=0.08
Max_132=0.70
C_132=0.121e-3

#66kV 185mm2 Cables
R_66=0.1640
X_66=0.07
Max_66=0.620
C_66=0.11e-3

Power_factor = 0.9
Theta = np.arccos(Power_factor)
P2Q = np.tan(Theta)  

#using excel sheet to allow fo iterations of demand rather than manual plugging in values

Mallorca_df = pd.read_excel(r'/Users/abdulsattaralkahala/Desktop/Balearic Data.xls',sheet_name='Mallorca')
Menorca_df=pd.read_excel(r'/Users/abdulsattaralkahala/Desktop/Balearic Data.xls',sheet_name='Minorca')
Ibiza_df=pd.read_excel(r'/Users/abdulsattaralkahala/Desktop/Balearic Data.xls',sheet_name='Ibiza')
Formentera_df=pd.read_excel(r'/Users/abdulsattaralkahala/Desktop/Balearic Data.xls',sheet_name='Formentera')
GenerationCapacity=pd.read_excel(r'/Users/abdulsattaralkahala/Desktop/Balearic Data.xls',sheet_name='Capacity Factors')

MallorcaDemand=Mallorca_df['Real']
#MallorcaCombinedCycle=Mallorca_df['Combined cycle']
#MallorcaSolarPV=Mallorca_df['Solar PV']
#MallorcaWastes=Mallorca_df['Wastes']
GFac=GenerationCapacity['Generation']
MenorcaDemand=Menorca_df['Real']
#MenorcaDiesel=Menorca_df['Diesel engines']
#MenorcaGasTurbine=Menorca_df['Gas turbine']
#MenorcaWind=Menorca_df['Wind']

IbizaDemand=Ibiza_df['Real']
#IbizaGasTurbine=Ibiza_df['Gas turbine']

FormenteraDemand=Formentera_df['Real']
#FormenteraSolarPV=Formentera_df['Solar pV']

#Create Network and Buses
net = pp.create_empty_network()

#Interconnector location
Morvedre_location= ( -0.235278, 39.641111)
Morvedre_Bus= pp.create_bus(net, vn_kv=hv, name = "Morvedre Interconnector", geodata=Morvedre_location)
Interconnector= pp.create_ext_grid(net, bus=Morvedre_Bus , vm_pu=1.02, name="Interconnector")

#Mallorca Buses
Santa_Ponsa_Location= (2.50895716,39.53767893)
Santa_Ponsa_hv_Bus = pp.create_bus(net,vn_kv=hv, name = "Santa Ponsa HV", geodata=Santa_Ponsa_Location)
Santa_Ponsa_mv_Bus = pp.create_bus(net,vn_kv=mv, name = "Santa Ponsa MV", geodata=Santa_Ponsa_Location)

Es_Bessons_location   = (3.15875,39.581939)
Es_Bessons_hv_Bus = pp.create_bus(net,vn_kv=hv, name = "Es Bessons HV", geodata=Es_Bessons_location)
Es_Bessons_mv_Bus = pp.create_bus(net,vn_kv=mv, name = "Es Bessons MV", geodata=Es_Bessons_location)

Llubi_location        = (3.039425,39.673758)
Llubi_Bus_hv=pp.create_bus(net, vn_kv=mv, name = "Llubi HV", geodata=Llubi_location)

Son_Reus_location     = (2.678805,39.650901)
Son_Reus_hv=pp.create_bus(net, vn_kv=mv, name = "Son Reus HV", geodata=Son_Reus_location)

Son_Orlandis_location = (2.744143,39.600178)
Son_Orlandis_hv=pp.create_bus(net, vn_kv=mv, name = "Son Orlandis HV", geodata=Son_Orlandis_location)

Valldurgent_location  = (2.549208,39.584228)
Valldurgent_hv=pp.create_bus(net, vn_kv=mv, name = "Valldurgent HV", geodata=Valldurgent_location)

Muretar_location      = (3.081611,39.809722)
Muretar_hv=pp.create_bus(net, vn_kv=mv, name = "Muretar HV", geodata=Muretar_location)

Cas_Tresoare_location     = (2.689469,39.567500)
Cas_Tresorer_hv=pp.create_bus(net, vn_kv=mv, name = "Cas Tresoare HV", geodata=Cas_Tresoare_location)

#Transformers

Santa_Ponsa_Transformer_HM = pp.create_transformer_from_parameters(net, hv_bus=Santa_Ponsa_hv_Bus, lv_bus=Santa_Ponsa_mv_Bus, sn_mva=400, vn_hv_kv=hv, vn_lv_kv=mv, vkr_percent=1, vk_percent=10, pfe_kw=400.*0.01, i0_percent=1)
Es_Bessons_Transformer_HM = pp.create_transformer_from_parameters(net, hv_bus=Es_Bessons_hv_Bus, lv_bus=Es_Bessons_mv_Bus, sn_mva=400, vn_hv_kv=hv,vn_lv_kv=mv, vkr_percent=1, vk_percent=10, pfe_kw=400*0.01, i0_percent=1)

#Lines

Morvedre_SantaPonsa_Length=247
pp.create_line_from_parameters(net,from_bus=Morvedre_Bus, to_bus=Santa_Ponsa_hv_Bus ,length_km=Morvedre_SantaPonsa_Length, r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Morvedre to Santa Ponsa ")


Santa_Ponsa_to_Valldurgent_length_1    = 8 
Santa_Ponsa_to_Valldurgent_length_2    = 8


pp.create_line_from_parameters(net,from_bus=Santa_Ponsa_hv_Bus, to_bus=Valldurgent_hv ,length_km=Santa_Ponsa_to_Valldurgent_length_1 ,  r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Santa Ponsa to Valldurgent 1 ")
pp.create_line_from_parameters(net,from_bus=Santa_Ponsa_hv_Bus, to_bus=Valldurgent_hv ,length_km=Santa_Ponsa_to_Valldurgent_length_1 ,  r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Santa Ponsa to Valldurgent 2 ")


Son_Reus_to_Valldurgent_length_1       = 19

pp.create_line_from_parameters(net,from_bus=Son_Reus_hv, to_bus=Valldurgent_hv ,length_km=Son_Reus_to_Valldurgent_length_1  , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Reus to Valldurgent 1 ")
pp.create_line_from_parameters(net,from_bus=Son_Reus_hv, to_bus=Valldurgent_hv ,length_km=Son_Reus_to_Valldurgent_length_1 , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Reus to Valldurgent 2 ")

Son_Reus_to_Son_Orlandis_length        = 12

pp.create_line_from_parameters(net,from_bus=Son_Reus_hv, to_bus=Son_Orlandis_hv ,length_km=Son_Reus_to_Son_Orlandis_length  , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Reus to Son Orlandis ")

Son_Reus_to_Llubi_length               = 32

pp.create_line_from_parameters(net,from_bus=Son_Reus_hv, to_bus=Llubi_Bus_hv ,length_km=Son_Reus_to_Llubi_length   , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Reus to Llubi")


Llubi_to_Es_Bessons_length_1           = 17
Llubi_to_Es_Bessons_length_2           = 17


pp.create_line_from_parameters(net,from_bus=Llubi_Bus_hv , to_bus=Es_Bessons_hv_Bus,length_km=Llubi_to_Es_Bessons_length_1   , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Llubi to Es Bessons 1 ")
pp.create_line_from_parameters(net,from_bus=Llubi_Bus_hv , to_bus=Es_Bessons_hv_Bus,length_km=Llubi_to_Es_Bessons_length_2  , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Llubi to Es Bessons 2")


Muretar_to_Llubi_length_1              = 18
Muretar_to_Llubi_length_2              = 18

pp.create_line_from_parameters(net,from_bus=Muretar_hv , to_bus=Llubi_Bus_hv,length_km=Muretar_to_Llubi_length_1    , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Muretar to Llubi 1  ")
pp.create_line_from_parameters(net,from_bus=Muretar_hv , to_bus=Llubi_Bus_hv,length_km=Muretar_to_Llubi_length_1   , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Muretar to Llubi 2")

Son_Orlandis_to_Cas_Tresorer_length_1  = 7
Son_Orlandis_to_Cas_Tresorer_length_2  = 7

pp.create_line_from_parameters(net,from_bus=Son_Orlandis_hv, to_bus= Cas_Tresorer_hv,length_km=Son_Orlandis_to_Cas_Tresorer_length_1  , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Orlandis to Cas Tresorer 1 ")
pp.create_line_from_parameters(net,from_bus=Son_Orlandis_hv, to_bus=Cas_Tresorer_hv ,length_km=Son_Orlandis_to_Cas_Tresorer_length_2 , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Orlandis to Cas Tresorer 2 ")


Son_Orlandis_to_Llubi_length           = 30
pp.create_line_from_parameters(net,from_bus=Son_Orlandis_hv, to_bus= Llubi_Bus_hv,length_km=Son_Orlandis_to_Llubi_length    , r_ohm_per_km =R_230, x_ohm_per_km =X_230,c_nf_per_km =C_230, max_i_ka=Max_230,name = "Son Orlandis to Llubi ")


#Minorca Islands
#Substations and Buses
Bosch_Location= (3.883702,39.932071)
Bosch_mv= pp.create_bus(net,vn_kv= mv, name = "Cala'n Bosch MV", geodata=Bosch_Location)

Ciudadela_Location=(3.855000,40.003333)
Ciudadela_mv= pp.create_bus(net,vn_kv= mv, name = "Ciudadela MV", geodata=Ciudadela_Location)

Mercadal_Location=(4.095683,39.977192)
Mercadal_mv= pp.create_bus(net,vn_kv= mv, name = "Mercadal MV", geodata=Mercadal_Location)

Dragonera_Location=(4.236667,39.891111)
Dragonera_mv= pp.create_bus(net,vn_kv= mv, name = "Dragonera MV", geodata=Dragonera_Location)

Mahon_Location= (4.259157,39.896648)
Mahon_mv= pp.create_bus(net,vn_kv= mv, name = "Mahon MV", geodata=Mahon_Location)

#Lines

Es_Bessons_Bosch_length=90
pp.create_line_from_parameters(net,from_bus=Es_Bessons_mv_Bus, to_bus=Bosch_mv,length_km=Es_Bessons_Bosch_length  , r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Es Bessons to Bosch")

Bosch_Ciudadela_length= 11
pp.create_line_from_parameters(net,from_bus=Bosch_mv, to_bus=Ciudadela_mv,length_km=Bosch_Ciudadela_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Bosch to Ciudadela ")

Cidadela_Mercadal_length= 23
pp.create_line_from_parameters(net,from_bus=Ciudadela_mv, to_bus=Mercadal_mv,length_km=Cidadela_Mercadal_length,r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Ciudadela to Mercadal ")

Cidadela_Dragonera_length= 41
pp.create_line_from_parameters(net,from_bus=Ciudadela_mv, to_bus=Dragonera_mv,length_km=Cidadela_Dragonera_length,r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Ciudadela to Dragonera ")

Mercadal_Dragonera_length= 18
pp.create_line_from_parameters(net,from_bus=Mercadal_mv, to_bus=Dragonera_mv,length_km= Mercadal_Dragonera_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Mercadal to Dragonera ")

Mahon_Dragonera_length= 3
pp.create_line_from_parameters(net,from_bus=Mahon_mv, to_bus=Dragonera_mv,length_km= Mahon_Dragonera_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Mahon to Dragonera 1 ")
pp.create_line_from_parameters(net,from_bus=Mahon_mv, to_bus=Dragonera_mv,length_km= Mahon_Dragonera_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132,name = "Mahon to Dragonera 2 ")


#Ibiza and Formentera Island

Torrent_location= (1.4325,38.935833)
Torrent_mv= pp.create_bus(net,vn_kv= mv, name = "TorrentMV", geodata=Torrent_location)

SanAntoni_location= (1.336111,38.957222)
SanAntoni_mv= pp.create_bus(net,vn_kv= mv, name = "SanAntoniMV", geodata=SanAntoni_location)

Ibiza_location= (1.378056,38.904444)
Ibiza_mv= pp.create_bus(net,vn_kv= mv, name = "IbizaMV", geodata=Ibiza_location)

Formentera_location= (1.463611,38.691944)
Formentera_mv= pp.create_bus(net,vn_kv= mv, name = "FormenteraMV", geodata=Formentera_location)

#Lines

Torrent_Santa_Ponsa_Length=115
pp.create_line_from_parameters(net, from_bus = Santa_Ponsa_mv_Bus, to_bus = Torrent_mv , length_km= Torrent_Santa_Ponsa_Length , r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132, name = 'Torrent to Santa Ponsa ')

Torrent_Formentera_Length=30
pp.create_line_from_parameters(net, from_bus = Torrent_mv, to_bus = Formentera_mv, length_km= Torrent_Formentera_Length , r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132, name = 'Torrent to Formentera ')

Torrent_SanAntoni_length=14
pp.create_line_from_parameters(net, from_bus = Torrent_mv, to_bus = SanAntoni_mv, length_km= Torrent_SanAntoni_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132, name = 'Torrent to San Antoni ')


Torrent_Ibiza_length=9
pp.create_line_from_parameters(net, from_bus = Torrent_mv, to_bus = Formentera_mv, length_km= Torrent_Ibiza_length , r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132, name = 'Torrent to Ibiza 1 ')
pp.create_line_from_parameters(net, from_bus = Torrent_mv, to_bus = Formentera_mv, length_km= Torrent_Ibiza_length, r_ohm_per_km =R_132, x_ohm_per_km =X_132,c_nf_per_km =C_132, max_i_ka=Max_132, name = 'Torrent to Ibiza 2')


for n in range (25):

    #P_Gen_Solar_Mallorca=MallorcaSolarPV.iloc[n]+140
    P_Gen_CombinedCycle_Mallorca=0.2*(MallorcaDemand.iloc[n]+MenorcaDemand.iloc[n]+IbizaDemand.iloc[n]+FormenteraDemand.iloc[n])
    #P_Gen_Waste_Mallorca=MallorcaWastes.iloc[n]


    #The ones that are currently hashtagged are for the current situation, the active ones are the for the solar farms
    P_CasTresorer=438.15*GFac.iloc[n]
    P_Llubi=904*GFac.iloc[n]
    P_Es_Bessons=169*GFac.iloc[n]
    P_Muretar=163.3*GFac.iloc[n]
    #P_SonReus=P_Gen_Waste_Mallorca+(P_Gen_Solar_Mallorca/2)
    #P_SonOrlandos= P_Gen_Waste_Mallorca+(P_Gen_Solar_Mallorca/2)
    #P_Muretar=P_Gen_CombinedCycle_Mallorca/3


    #CasTresorer_Generation = pp.create_sgen(net, bus = Cas_Tresorer_hv, p_mw = P_CasTresorer, q_mvar = P_CasTresorer *P2Q, name = "Tresoare PV Station")
    #Son_Reus_Generation = pp.create_sgen(net, bus = Son_Reus_hv, p_mw = P_SonReus, q_mvar = P_SonReus *P2Q, name = "Son Reus PV and Waste Station")
    #Muretar_Generation  = pp.create_sgen(net, bus = Muretar_hv , p_mw = P_Muretar, q_mvar =  P_Muretar*P2Q , name = "Muretar Power Station" )
    #SonOrlandos_Generation  = pp.create_sgen(net, bus = Son_Orlandis_hv , p_mw =P_SonOrlandos, q_mvar = P_SonOrlandos*P2Q , name = "SonOrlandos PV Station" )

    Muretar_PV_Gen=pp.create_sgen(net,bus=Muretar_hv, p_mw = P_Muretar, q_mvar =   P_Muretar*P2Q , name = "Muretar Solar Station" )
    Es_Bessons_PV_Gen=pp.create_sgen(net,bus=Es_Bessons_mv_Bus, p_mw =  P_Es_Bessons, q_mvar =    P_Es_Bessons*P2Q , name = "Es Bessons Solar Station" )
    Llubi_PV_Gen=pp.create_sgen(net,bus= Llubi_Bus_hv,p_mw= P_Llubi,q_mvar =   P_Llubi*P2Q,name = "Llubi Solar Station")
    CasTresorer_PV_Gen=pp.create_sgen(net,bus= Cas_Tresorer_hv,p_mw= P_CasTresorer,q_mvar = P_CasTresorer*P2Q,name = "Cas Tresorer Solar Station")


    #Create Mallorca Loads according to total demand and population density
    P_Demand_Mallorca=MallorcaDemand.iloc[n]
    Population_CasTresorer=0.517657733
    Population_SantaPonsa=0.058670503
    Population_Valldurgent=0.015702459
    Population_SanReus=0.024319281
    Population_SanOrlandis=0.070627585
    Population_EsBessons=0.113053241
    Population_Llubi=0.074063823
    Population_Murterar=0.040761664

    pp.create_load(net, bus= Cas_Tresorer_hv, p_mw=P_Demand_Mallorca *Population_CasTresorer,q_mvar= P_Demand_Mallorca *Population_CasTresorer*P2Q, name="Cas Tresorer Demand")
    pp.create_load(net, bus= Santa_Ponsa_mv_Bus, p_mw=(P_Demand_Mallorca *Population_SantaPonsa),q_mvar=P_Demand_Mallorca *Population_SantaPonsa, name="Santa Ponsa Demand ")
    pp.create_load(net, bus= Valldurgent_hv, p_mw=P_Demand_Mallorca *Population_Valldurgent,q_mvar= P_Demand_Mallorca *Population_Valldurgent*P2Q, name="Valldurgent Demand")
    pp.create_load(net, bus= Son_Reus_hv, p_mw=P_Demand_Mallorca *Population_SanReus,q_mvar= P_Demand_Mallorca *Population_SanReus, name="San Reus Demand")
    pp.create_load(net, bus= Son_Orlandis_hv, p_mw=P_Demand_Mallorca *Population_SanOrlandis,q_mvar= P_Demand_Mallorca *Population_SanOrlandis*P2Q, name="San Orlandis Demand")
    pp.create_load(net, bus= Es_Bessons_mv_Bus, p_mw=P_Demand_Mallorca *Population_EsBessons,q_mvar= P_Demand_Mallorca *Population_EsBessons *Population_EsBessons*P2Q, name="Es Bessons Demand")
    pp.create_load(net, bus= Llubi_Bus_hv, p_mw=P_Demand_Mallorca *Population_Llubi,q_mvar= P_Demand_Mallorca *Population_Llubi*P2Q, name="Llubi Demand")
    pp.create_load(net, bus= Muretar_hv, p_mw=P_Demand_Mallorca *Population_Murterar,q_mvar= P_Demand_Mallorca *Population_Murterar*P2Q, name="Murterar Demand")

    #Create Minorca Generation at 1.30PM

    #P_Gen_NonRenewable=MenorcaDiesel.iloc[n]+MenorcaGasTurbine.iloc[n]
    #P_Gen_Renewable=MenorcaWind.iloc[n]
    #P_Mahon= P_Gen_NonRenewable
    #P_Ciudadela=P_Gen_Renewable
    #Mahon_Generation=pp.create_sgen(net, bus = Mahon_mv, p_mw = P_Mahon, q_mvar = P_Mahon *P2Q, name = "Mahon Power Station")
    #Ciudadela_Generation=pp.create_sgen(net, bus = Ciudadela_mv, p_mw = P_Ciudadela, q_mvar = P_Ciudadela *P2Q, name = "Ciudadela Power Station")

    P_Gen_Bosch=161*GFac.iloc[n]
    P_Gen_Mahon=181*GFac.iloc[n]
    P_Gen_Ciudadela=37*GFac.iloc[n]

    Bosch_PV_Gen=pp.create_sgen(net,bus=Bosch_mv, p_mw = P_Gen_Bosch , q_mvar =  P_Gen_Bosch *P2Q, name = "Bosch Solar Station" )
    Mahon_PV_Gen=pp.create_sgen(net,bus=Mahon_mv, p_mw = P_Gen_Mahon , q_mvar =  P_Gen_Mahon*P2Q, name = "Dragonera Solar Station" )
    Ciudadela_PV_Gen=pp.create_sgen(net,bus= Ciudadela_mv, p_mw =P_Gen_Ciudadela , q_mvar = P_Gen_Ciudadela*P2Q, name = "Ciudadela Solar Station" )


    #Create Minorca Loads according to total demand and population density
    P_Demand_Minorca=MenorcaDemand.iloc[n]
    Population_Dragonera=0.46331253
    Population_Bosch=0.11210210
    Population_Ciudadela= 0.37064360
    Population_Mercadal=0.05394178

    pp.create_load(net, bus= Dragonera_mv, p_mw=P_Demand_Minorca*Population_Dragonera,q_mvar= P_Demand_Minorca*Population_Dragonera*P2Q, name="Dragonera Demand")
    pp.create_load(net, bus= Bosch_mv, p_mw=P_Demand_Minorca*Population_Bosch,q_mvar= P_Demand_Minorca*Population_Bosch*P2Q, name="Bosch Demand")
    pp.create_load(net, bus= Ciudadela_mv, p_mw=P_Demand_Minorca*Population_Ciudadela,q_mvar= P_Demand_Minorca*Population_Ciudadela*P2Q, name="Ciudadela Demand")
    pp.create_load(net, bus= Mercadal_mv, p_mw=P_Demand_Minorca*Population_Mercadal,q_mvar= P_Demand_Minorca*Population_Mercadal*P2Q, name="Mercadal Demand")


    #P_Ibiza= IbizaGasTurbine.iloc[n]
    #Ibiza_Generation=pp.create_sgen(net, bus = Ibiza_mv ,p_mw = P_Ibiza, q_mvar = P_Ibiza*P2Q, name = "Ibiza Power Station")

    #Create Formentera Generation at 1.30PM
    #P_Renewable_Formentera=FormenteraSolarPV.iloc[n]
    #Formentera_Generation=pp.create_sgen(net, bus = Formentera_mv, p_mw =P_Renewable_Formentera, q_mvar = P_Renewable_Formentera*P2Q, name = "Formentera Solar PV Farm")

    #Create Ibiza Demand at 1.30PM
    P_Demand_Ibiza=IbizaDemand.iloc[n]

    Torrent_Population= 0.485586
    Ibiza_Population= 0.336567
    SanAntoni_Population=0.177847

    pp.create_load(net, bus= Torrent_mv, p_mw= P_Demand_Ibiza*Torrent_Population,q_mvar=P_Demand_Ibiza*Torrent_Population*P2Q, name="Torrent Demand")
    pp.create_load(net, bus= Ibiza_mv, p_mw= P_Demand_Ibiza*Ibiza_Population,q_mvar= P_Demand_Ibiza*Ibiza_Population*P2Q, name="Ibiza Demand")
    pp.create_load(net, bus= SanAntoni_mv, p_mw= P_Demand_Ibiza*SanAntoni_Population,q_mvar=P_Demand_Ibiza*SanAntoni_Population*P2Q, name="San Antoni Demand")

   
    #Formentera Renewable Gen
    
    P_Formentera_Gen=382*GFac.iloc[n]
    Formentera_PV_Gen=pp.create_sgen(net,bus=Formentera_mv, p_mw = P_Formentera_Gen , q_mvar = P_Formentera_Gen *P2Q, name = "Formentera Solar Station" )

    P_Demand_Formentera=FormenteraDemand.iloc[n]
    pp.create_load(net, bus= Formentera_mv, p_mw=P_Demand_Formentera,q_mvar= P_Demand_Formentera*P2Q, name="Formentera Demand")

    pp.runpp(net)
   
    net.res_line.to_excel("Solar PV without Battery.xlsx")
    net.line.to_excel("Line Names.xlsx")
    net.bus.to_excel("Names.xlsx")
    






